# Dataset Properati

### Limpieza

Descargamos el dataset de Properati con propiedades en venta en Argentina de los últimos 6 meses. Cambiar la fecha en la URL de estar desactualizada. Ejercicio: armar un dataset con avisos de los últimos 12 meses.

In [3]:
import requests

url = 'https://www.properati.com.ar/static/data/AR/properati-AR-2018-03-01-properties-sell-six_months.csv.gz'

r = requests.get(url)

with open('datos_properati.csv.gz', 'wb') as f:  
    f.write(r.content)

Leemos el archivo descargado.

In [86]:
import pandas as pd

df = pd.read_csv('datos_properati.csv.gz', parse_dates=['created_on'])
df.set_index('id', inplace=True)

Nos deshacemos del algunas columnas.

In [87]:
df.drop(columns=[
    'operation', # son todas propiedades en venta
    'place_name', # ver place_with_parent_names...
    'place_with_parent_names', # vamos a estimar el lugar usando las coordenadas
    'country_name', # son todas propiedades de Argentina
    'state_name', # en algún momento nos vamos a quedar con propiedades de CABA solamente
    'geonames_id', # no hace falta
    'lat-lon', # ya viene lat y lon por separado
    'price', # vamos a usar price_aprox_usd
    'currency', # ídem
    'price_aprox_local_currency', # ídem
    'price_usd_per_m2', # este dato puede ser reconstruido luego con mayor calidad
    'price_per_m2', # ídem
    'properati_url', # no hace falta
    'image_thumbnail', # ídem
], inplace=True)

Solo por diversión, cambiamos nombres de columnas.

In [88]:
df.rename(columns={
    'price_aprox_usd':'price_usd',
    'surface_total_in_m2':'surface_total',
    'surface_covered_in_m2':'surface_covered'
}, inplace=True)

Ahora nos vamos a quedar con propiedades:

* geolocalizadas,
* que sean casas o departamentos,
* que informen al menos una superficie (cubierta o total),
* que informen cantidad de ambientes,
* que informen precio de venta.

In [89]:
geo        =  df.lat.notnull() & df.lon.notnull()
tipo       =  df.property_type.isin(['house', 'apartment'])
superficie =  df.surface_covered.notnull() | df.surface_total.notnull() 
ambientes  =  df.rooms.notnull()
precio     =  df.price_usd.notnull()

df = df[geo & tipo & superficie & ambientes & precio]

#### Un poco de GeoPandas

Usamos el [dataset de barrios](https://data.buenosaires.gob.ar/dataset/barrios) de BA Data para determinar por medio de las coordenas geográficas a qué barrio pertenece cada propiedad.

In [90]:
import geopandas as gpd
from shapely.geometry import Point

barrios = gpd.read_file('barrios/barrios_badata.shp').to_crs(epsg=4326)
barrios.drop(columns=['AREA', 'PERIMETRO'], inplace=True)
barrios.rename(columns={'COMUNA':'comuna', 'BARRIO':'barrio'}, inplace=True)

In [91]:
df = gpd.GeoDataFrame(df, crs={'init':'epsg:4326'})
df.geometry = df.apply(lambda row: Point(row.lon, row.lat), axis=1)
df = gpd.sjoin(df, barrios, how='inner', op='intersects')

#### Registros duplicados

Tiramos propiedades repetidas, según todas las columnas exceptuando `created_on`. Las inmobiliarias suelen volver a publicar sus propiedades para que aparezcan múltiples veces en el sitio. A veces lo hacen en momentos diferentes, en este caso la fecha de creación haría que dos filas de la tabla que coincidan en el resto de sus atributos no sean consideradas la misma propiedad, por eso no la usamos para la comparación.

In [92]:
df.drop_duplicates([
    'property_type',
    'place_name',
    'state_name',
    'lat',
    'lon',
    'price_usd',
    'surface_total',
    'surface_covered',
    'floor',
    'rooms',
    'expenses',
    'description',
    'title',
], inplace=True)

Queremos filtrar las propiedades con coordenadas que se repiten mucho. Cuando no se cuenta con las coordenadas reales las inmobiliarias suelen asignar centroides de barrios o puntos de interés como el Obelisco.

Nos quedamos con las propiedades con coordenadas que se repitan hasta 5 veces.

In [98]:
conteo_puntos = df.groupby(['lat', 'lon']).size()
conteo_puntos.name = 'conteo'

df = df.join(conteo_puntos, on=['lat', 'lon'])

df = df[df.conteo <= 5]

Un poco más de limpieza todavía.

In [100]:
df.drop(columns=['index_right', 'conteo'], inplace=True)

#### Imputación de valores

Si la superficie total es inexistente o cero, la imputamos desde la superficie cubierta, y viceversa.

In [101]:
# total
mal = df[~(df.surface_total > 0)]
df.loc[mal.index, 'surface_total'] = mal.surface_covered

# cubierta
mal = df[~(df.surface_covered > 0)]
df.loc[mal.index, 'surface_covered'] = mal.surface_total

Si la superficie total es menor a la cubierta, lo cual no tiene sentido, intercambiamos los valores.

In [102]:
mal = df[df.surface_total < df.surface_covered]
df.loc[mal.index, 'surface_total'], df.loc[mal.index, 'surface_covered'] = mal.surface_covered, mal.surface_total

### Separación del dataset en desarrollo y prueba

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=1)

### Desempeño

Para calcular el error, vamos a usar el error relativo.

In [60]:
def error_relativo(precio_real, precio_predicción):
    e = np.abs(precio_real - precio_predicción) / precio_real * 100
    return e.round(1)

In [ ]:
test['predicción'] = modelo.predict(test)
test['error'] = error_relativo(test.price_usd, test.predicción)

Para reportar el error, vamos a usar la mediana. Cuando hay *outliers*, la mediana funciona mejor que el promedio.

In [ ]:
test.error.median()

¿Cómo te fue? El valuador de Properati consigue un error del 15%. Esto quiere decir que para *la mitad de las propiedades* en venta en CABA, el error de la estimación es del 15% o menos. 